# Basic Exploratory Data Analysis On Entire Dataset

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import mysql.connector
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from pandas.core import datetools
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.cluster import KMeans

sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

/home/gspc/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def create_connection():
    cnx2 = mysql.connector.connect(host='localhost',
                                   user='root', password='MyNewPass',
                                   database='Weather_Data')
    return cnx2

In [3]:
def select_data():
    conn = create_connection()
    cursor = conn.cursor()
    print('Extracting Data')
    query = "Select * from weather where Observation_date <= '2018-06-01 00:00:00';"
    cursor.execute(query, )
    result = cursor.fetchall()
    print('Extracted Data')
        
    cursor.close()
    conn.close()
    
    return result
    

In [4]:
def null_values(data):
    print(data.isnull().sum())

In [5]:
def change_structure(data):
    cols = ['date', 'rain', 'temp', 'wetb', 'dewpt','vappr', 'rhum', 'msl', 'wdsp', 'wddir','height','latitude', 'longitude', 'station','county']
    data = pd.DataFrame(data, columns=cols)
    
    null_values(data)
    data.fillna(0, inplace=True)
    return data

In [6]:
def split_time(data):
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year.astype(np.uint16)
    data['month'] = data['date'].dt.month.astype(np.uint8)
    data['day'] = data['date'].dt.day.astype(np.uint8)
    data['hour'] = data['date'].dt.hour.astype(np.uint8)
    
    return data

In [7]:
def aggregate_data(data):
    print('Aggregating Data')
    data = data.set_index('date').groupby(pd.Grouper(freq='H')).mean().dropna()
    
    print(len(data),' observations available for training')
    return data

In [8]:
data = select_data()
data = change_structure(data)
data = split_time(data)
data = aggregate_data(data)

Extracting Data
Extracted Data
date         0
rain         0
temp         0
wetb         0
dewpt        0
vappr        0
rhum         0
msl          0
wdsp         0
wddir        0
height       0
latitude     0
longitude    0
station      0
county       0
dtype: int64
Aggregating Data
266617  observations available for training


In [9]:
temp = data

In [10]:
grouping = ['year','month','day']
cols = ['rain']

temp = temp.groupby(grouping).agg({'rain': ['min', 'max']})
#print(temp)
#temp.columns = new_column_names
t = temp[temp[('rain','max')] >= .1]
t

rain     
                min  max
year month day          
1988 1     1   0.00 0.73
           2   0.09 0.82
           3   0.00 0.84
           4   0.00 0.67
           5   0.00 1.36
           6   0.00 0.96
           7   0.00 0.72
           8   0.00 0.84
           9   0.00 0.96
           10  0.00 0.42
           11  0.00 0.73
           12  0.05 1.60
           13  0.02 0.48
           14  0.00 0.26
           15  0.00 0.19
           16  0.00 0.26
           17  0.00 0.33
           18  0.00 2.86
           19  0.05 1.45
           20  0.01 0.22
           21  0.00 1.09
           22  0.00 0.98
           23  0.00 1.64
           24  0.00 0.71
           25  0.00 0.36
           26  0.00 0.21
           27  0.00 0.29
           28  0.00 0.67
           29  0.05 0.54
           30  0.01 0.30
...             ...  ...
2018 4     14  0.00 0.25
           15  0.05 0.43
           16  0.00 1.47
           17  0.00 1.65
           18  0.01 0.14
           21  0.00 0.57
           22  0.00 0.43
           23  0.00 0.20
           24  0.00 1.34
           25  0.00 0.34
           26  0.02 0.28
           27  0.00 0.32
           28  0.00 0.19
           29  0.00 0.27
     5     1   0.00 0.61
           2   0.00 0.12
           3   0.00 0.37
           8   0.00 0.29
           9   0.00 0.71
           10  0.00 0.17
           11  0.00 1.40
           12  0.00 0.21
           13  0.00 0.13
           15  0.00 0.13
           19  0.00 0.13
           20  0.09 0.61
           21  0.00 1.22
           24  0.00 0.16
           27  0.00 0.78
           31  0.00 0.15

[7888 rows x 2 columns]

In [16]:
temp[temp[('rain','max')] >= 2]

rain     
                min  max
year month day          
1988 1     18  0.00 2.86
     7     12  0.00 2.40
           28  0.00 3.20
     8     8   0.00 2.68
           13  0.00 2.02
           31  0.00 2.22
     10    11  0.00 3.26
           21  0.16 2.03
1989 6     9   0.24 2.18
     8     14  0.00 2.12
           25  0.00 2.08
           30  0.00 2.24
     10    4   0.00 2.16
           27  0.60 2.60
     12    13  0.00 2.01
           16  0.42 2.30
           24  0.00 2.12
1990 1     23  0.03 2.26
           25  0.01 2.58
     7     4   0.07 2.16
           29  0.00 2.36
     10    2   0.07 3.44
           15  0.14 2.34
1991 1     1   0.00 2.10
     7     11  0.00 2.48
     9     15  0.00 2.20
     10    30  0.00 2.54
           31  0.02 3.69
     11    12  0.04 2.71
1992 6     2   0.01 2.61
...             ...  ...
2012 4     16  0.00 2.18
           17  0.05 2.42
     6     28  0.01 2.91
     10    17  0.18 2.06
2013 1     18  0.18 2.23
     10    16  0.01 3.06
     12    14  0.00 2.10
           18  0.01 4.27
2014 1     26  0.03 2.84
           31  0.01 2.57
     2     12  0.09 2.50
           14  0.00 2.00
     3     20  0.01 2.07
     10    5   0.02 2.74
           28  0.00 2.27
     11    13  0.02 2.16
           14  0.01 2.95
2015 7     26  0.08 2.22
     9     11  0.54 2.23
     11    18  0.01 2.04
     12    3   0.00 2.67
           5   0.44 2.15
           12  0.16 2.67
           30  0.01 2.34
2016 1     26  0.05 2.16
2017 6     9   0.00 2.19
           10  0.00 2.13
     9     27  0.01 2.18
     11    22  0.04 2.54
2018 1     24  0.08 2.27

[184 rows x 2 columns]

In [17]:
temp = data
temp[(temp['month'] == 1) & (temp['year'] == 2018) & (temp['day'] == 24 )]

,rain,temp,wetb,dewpt,vappr,rhum,msl,wdsp,wddir,height,latitude,longitude,year,month,day,hour
date,,,,,,,,,,,,,,,,
2018-01-24 00:00:00,1.03,10.37,9.54,8.68,11.25,89.00,992.89,23.61,190.00,59.87,53.21,-8.20,2018,1,24,0
2018-01-24 01:00:00,1.85,10.34,9.48,8.61,11.21,88.70,990.91,24.09,199.57,59.87,53.21,-8.20,2018,1,24,1
2018-01-24 02:00:00,2.27,9.46,8.63,7.69,10.55,88.52,990.10,24.13,207.83,59.87,53.21,-8.20,2018,1,24,2
2018-01-24 03:00:00,1.84,8.64,7.60,6.33,9.64,85.17,990.20,23.87,219.57,59.87,53.21,-8.20,2018,1,24,3
2018-01-24 04:00:00,1.14,7.74,6.37,4.56,8.51,80.39,990.62,23.70,230.87,59.87,53.21,-8.20,2018,1,24,4
2018-01-24 05:00:00,0.59,7.17,5.68,3.66,7.96,78.17,991.17,22.00,232.61,59.87,53.21,-8.20,2018,1,24,5
2018-01-24 06:00:00,0.16,6.70,5.31,3.42,7.83,79.39,991.50,19.17,228.26,59.87,53.21,-8.20,2018,1,24,6
2018-01-24 07:00:00,0.10,6.78,5.40,3.56,7.90,79.65,991.75,17.83,225.22,59.87,53.21,-8.20,2018,1,24,7
2018-01-24 08:00:00,0.24,6.55,5.28,3.56,7.88,80.91,992.08,17.83,223.48,59.87,53.21,-8.20,2018,1,24,8
